base_app
- new_scene
- directory
- image_dir
- get_image_dir
- progress
- invalidate
- step
- fieldnames
- get_field
- add_field
- actions
- add_action
- run_action
- traits
- add_trait
- controls
- prepare
- _add_default_fields
- add_custom_property
- add_custom_properties
- _update_scene_properties
- settings_str
- custom_properties
- info
- debug
- scene_summary
- show
- status
- run_step
- play
- pause
- running
- record_frame
- benchmark
- config_recording

App
- prepare
- add_scalar
- editable_float
- editable_int
- editable_values_dict
- get_editable_value
- set_editable_value

LearningApp
- prepare
- set_learning_rate
- set_data
- _init_numpy_iterators
- _init_tf_pipeline
- add_objective
- step
- optimization_step
- on_training_set_end
- validation_step
- base_feed_dict
- _feed_dict
- view_reader
- _view_dataset
- view
- viewed_batch
- view_batch
- save_model
- load_model
- model_scope
- add_field

ControlTraining
- add_all_fields
- load_checkpoints
- action_save_model
- step
- infer_all_frames
- infer_scalars

In [ ]:
from phi.tf.session import Session
from phi.data.fluidformat import Scene
from phi.physics.world import World
from phi.physics.domain import Domain
from src.control.pde.incompressible_flow import IncompressibleFluidPDE
from src.control.hierarchy import PDEExecutor
from src.control.sequences import StaggeredSequence
import tensorflow as tf

In [ ]:
batch_size = 16
domain = Domain([64, 64])
dt = 1.0
step_count = 16
obs_loss_frames = [step_count]
trainable_networks = ['CFE', 'OP2', 'OP4', 'OP8', 'OP16']
learning_rate = 5e-4
reg_scale = 1.0

In [ ]:
config = tf.ConfigProto()
scene = Scene.create("data", None, count=1, mkdir=True, copy_calling_script=False)
session = Session(scene, session=tf.Session(config=config))
world = World(batch_size=batch_size)
pde = IncompressibleFluidPDE(domain, dt)
pde.create_pde(world, 'CFE' in trainable_networks, True)
world.state = pde.placeholder_state(world, 0)
target_state = pde.placeholder_state(world, step_count * dt)
in_states = [world.state] + [None] * (step_count - 1) + [target_state]

executor = PDEExecutor(world, pde, target_state, trainable_networks, dt)
sequence = StaggeredSequence(step_count, executor)
sequence.execute()

all_states = [frame.worldstate for frame in sequence if frame is not None]

In [ ]:
loss = 0
target_loss = pde.target_matching_loss(target_state, sequence[-1].worldstate)
reg = pde.total_force_loss([state for state in all_states if state is not None])
loss += target_loss
for frame in obs_loss_frames:
    supervised_loss = pde.target_matching_loss(in_states[frame], sequence[frame].worldstate)
    loss += supervised_loss

In [ ]:
all_optimizers = []
optimizer = tf.train.AdamOptimizer(learning_rate)
optim_function = loss + reg

node = optimizer.minimize(optim_function, var_list=None)
all_optimizers.append(node)

In [ ]:
def info(message):
    print(str(message))

In [ ]:
session.initialize_variables()

supervised_checkpoints = {'OP%d' % n: 'ckpts/supervised/OP%d_1000' % n for n in [2, 4, 8, 16]}
supervised_checkpoints['CFE'] = 'ckpts/CFE/CFE_1000'
executor.load(step_count, supervised_checkpoints, preload_n=True, session=session, logf=info)

In [ ]:
def optimization_step(optim_nodes, log_loss=None):
    try:
        optim_nodes = list(optim_nodes)
    except:
        optim_nodes = [optim_nodes]


In [ ]:
import tqdm

for _ in tqdm.tqdm(range(1000)):
    world.step(dt=dt)